In [56]:
import importlib.machinery
pyfor = importlib.machinery.SourceFileLoader('pyfor', '/home/bryce/Programming/PyFor/pyfor/__init__.py').load_module()
from scipy.spatial import cKDTree
import numpy as np
from scipy.stats import linregress
from numba import jit
%load_ext line_profiler

pc = pyfor.cloud.Cloud('/home/bryce/Programming/PyFor/pyfortest/data/test.las')
pc.normalize(0.5)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [57]:
# Some test data
xyz = pc.las.points[["x", "y", "z"]].values
tree = cKDTree(xyz[:,0:2])

tree.query(xyz[0, 0:2])

dist, inds = tree.query(xyz[0, 0:2], k = 12)

X = xyz[inds][:, 0:2]
y = xyz[inds][:,2]

In [12]:
@jit
def get_weights(x_i_dists):
    """

    :param x_i_dists:
    :return: An array of weights for each of the 12 neighbors.
    """
    # We will assume anything passed to this function
    # is a neighbor
    d_ij = x_i_dists
    d_max = np.max(x_i_dists)
    return ((1 - (d_ij/ d_max))**3)**3

In [31]:
@jit
def weighted_regression(X, y, weights, x_i_dists):
    """
    Weighted regression (without intercept?)

    :param X:
    :param y:
    :param weights:
    :param x_i_dists:
    :return: Returns fitted values
    """
    weights = get_weights(x_i_dists)
    weights = np.diag(weights)

    beta_hat =  np.linalg.inv(X.T @ weights @ X) @ X.T @ weights @ y

    return (beta_hat, y - X @ beta_hat)

In [41]:
@jit
def robust_weights(resids):
    s = np.median(resids)
    
    mask = np.where(resids / (6 * s) < 1)
    
    resids[mask] = 1 - ((resids[mask] / (6 * s))**2)**2
    resids[np.logical_not(resids / (6*s) < 1)] = 0
    return resids

In [45]:
def rlwr_neighborhood(dists, X, y):
    """

    :param index: The index of the XY array to do rlwr on
    :return:
    """
    weights = get_weights(dists)

    beta_hat, resid = weighted_regression(X, y, weights, dists)

    r_weights = robust_weights(resid)
    new_weights = weights * r_weights
    beta_hat, resid = weighted_regression(X, y, new_weights, dists)

    # Calculate interpolation error
    g_x_i = beta_hat @ X[0]
    return g_x_i

In [59]:
def chen_interpolate(xyz, index,  kd_tree, k=12):
    """
    For a given LiDAR point, interpolates using the algorithm outlined in Chen et. al. (2017)

    :param xy_coord: A 1x2 array describing the x y coordinate of the LiDAR point
    :param kd_tree: The kd_tree object to query, this should be a tree constructed on the 2D space LiDAR coordinates \
    for the area of interest
    :return:
    """
    # Retrieve k nearest distances and their indices
    distances, indices = kd_tree.query(xyz[index, 0:2], k = k)

    X = xyz[indices][:, 0:2]
    y = xyz[indices][:, 2]
    g_x_i = rlwr_neighborhood(distances, X, y)

    # Calculate interpolation error
    e_i = xyz[indices][0][2] - g_x_i

    return e_i

In [60]:
[chen_interpolate(xyz, i, tree) for i in range(xyz.shape[0])]

[0.023303347751493675,
 -0.011659835672503505,
 0.058727210927770557,
 -0.60330340659015746,
 -0.020115858390965968,
 0.45841058654241351,
 -0.066458578258789203,
 0.27848362324760956,
 -0.72050560131668817,
 1.428156902715557,
 0.064432836771004531,
 0.74953961631280208,
 -0.13915623337032912,
 -3.3942838445678376,
 0.046285302266483086,
 0.70970089215785492,
 0.3998498958349046,
 -0.41612926989796506,
 -0.13350718656090521,
 -2.9177472206950483,
 -1.4944689938425881,
 0.023690489307057305,
 0.063012349009511581,
 -0.73071547344329701,
 -0.21576571255923227,
 -0.28282306641341393,
 -0.26727670952681137,
 1.1446699771067301,
 0.22205822534857589,
 0.17558571963104441,
 0.50316915526985895,
 -2.7273407475650515,
 -3.6415257823467186,
 -0.074563979357492371,
 0.75151328232135484,
 -1.1642502781749045,
 1.6376459389924776,
 0.051180359497664085,
 -0.15705455243590905,
 -0.0092682190239656848,
 0.00059728085994947833,
 -0.13803857594729152,
 -0.11578456610442345,
 0.20408726362745711,
 0.7